# Парсинг данных. Практика 2

Скачаем книги с сайта http://books.toscrape.com/catalogue/page-1.html

Мы хотим создать базу данных со следующей информацией:

* Ссылка на книгу 
* Стоимость 
* Название
* Оценка (кол-во звезд)

In [1]:
import requests  
from bs4 import BeautifulSoup

In [2]:
url = 'http://books.toscrape.com/catalogue/page-1.html'
response = requests.get(url)
response

<Response [200]>

In [3]:
soup = BeautifulSoup(response.content, 'html')

In [4]:
soup.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

In [5]:
books = soup.find_all('article', {'class' : 'product_pod'})
books[0]

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [8]:
books[0].find('p', {'class': 'price_color'}).text

'£51.77'

In [9]:
books[0].find_all('a')[0].get('href')

'a-light-in-the-attic_1000/index.html'

In [10]:
books[0].find('h3').find('a').get('title')

'A Light in the Attic'

In [13]:
books[7].find('p').get('class')[1]

'Three'

In [14]:
def get_page(p):
    url = f'http://books.toscrape.com/catalogue/page-{p}.html'
    response = requests.get(url)
    tree = BeautifulSoup(response.content, 'html.parser')
    books = tree.find_all('article', {'class' : 'product_pod'})
    
    info = []
    
    for book in books:
        info.append({'price': book.find('p', {'class': 'price_color'}).text,
                     'href': 'https://books.toscrape.com/catalogue/' + book.find('h3').find('a').get('href'),
                     'title': book.find('h3').find('a').get('title'),
                    'rating': book.find('p').get('class')[1]})
                     
    return info

In [15]:
from tqdm import tqdm

In [20]:
import time
infa = []

for p in tqdm(range(1,51)):
    try:
        infa.extend(get_page(p))
        time.sleep(1)
    except:
        print(p)

100%|██████████| 50/50 [01:12<00:00,  1.45s/it]


In [21]:
infa

[{'price': '£51.77',
  'href': 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
  'title': 'A Light in the Attic',
  'rating': 'Three'},
 {'price': '£53.74',
  'href': 'https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
  'title': 'Tipping the Velvet',
  'rating': 'One'},
 {'price': '£50.10',
  'href': 'https://books.toscrape.com/catalogue/soumission_998/index.html',
  'title': 'Soumission',
  'rating': 'One'},
 {'price': '£47.82',
  'href': 'https://books.toscrape.com/catalogue/sharp-objects_997/index.html',
  'title': 'Sharp Objects',
  'rating': 'Four'},
 {'price': '£54.23',
  'href': 'https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
  'title': 'Sapiens: A Brief History of Humankind',
  'rating': 'Five'},
 {'price': '£22.65',
  'href': 'https://books.toscrape.com/catalogue/the-requiem-red_995/index.html',
  'title': 'The Requiem Red',
  'rating': 'One'},
 {'price': '£33.34',
  'href': 'https:

In [22]:
import pandas as pd

df = pd.DataFrame(infa)
print(df.shape)
df.head()

(1000, 4)


,price,href,title,rating
0,£51.77,https://books.toscrape.com/catalogue/a-light-i...,A Light in the Attic,Three
1,£53.74,https://books.toscrape.com/catalogue/tipping-t...,Tipping the Velvet,One
2,£50.10,https://books.toscrape.com/catalogue/soumissio...,Soumission,One
3,£47.82,https://books.toscrape.com/catalogue/sharp-obj...,Sharp Objects,Four
4,£54.23,https://books.toscrape.com/catalogue/sapiens-a...,Sapiens: A Brief History of Humankind,Five


In [23]:
df.rating.value_counts()

rating
One      226
Three    203
Five     196
Two      196
Four     179
Name: count, dtype: int64

In [24]:
def stars_num(x):
    if  x == 'One':
        return 1
    elif x == 'Two':
        return 2
    elif x == 'Three':
        return 3
    elif x == 'Four':
        return 4
    else:
        return 5

In [25]:
df['rating_num'] = df['rating'].apply(stars_num)

In [26]:
df.sample(5)

,price,href,title,rating,rating_num
113,£36.72,https://books.toscrape.com/catalogue/codename-...,"Codename Baboushka, Volume 1: The Conclave of ...",Four,4
951,£37.46,https://books.toscrape.com/catalogue/sense-and...,Sense and Sensibility,One,1
228,£25.38,https://books.toscrape.com/catalogue/through-t...,Through the Woods,Two,2
664,£20.02,https://books.toscrape.com/catalogue/no-one-he...,No One Here Gets Out Alive,Five,5
25,£54.64,https://books.toscrape.com/catalogue/birdsong-...,Birdsong: A Story in Pictures,Three,3


In [27]:
df.to_csv('books_parsed.csv', index=False)

In [28]:
df.to_excel('books_parsed.xlsx', index=False)

## Парсинг – задание

По аналогии вам предстоит собрать данные с сайта https://quotes.toscrape.com/. 

Нужно получить цитату, автора и теги, связанные с этой цитатой. 

Напишите программу, где пользователь вводит тег, а ему выводятся все цитаты, у которых есть такой тег (например, 'truth').

In [40]:
url2 = 'https://quotes.toscrape.com/page/1/' # всего 10 страниц: 1 - 10

In [38]:
response2 = requests.get(url2)
response2

<Response [200]>

In [39]:
soup2 = BeautifulSoup(response2.content, 'html')

In [36]:
soup2

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="t

In [43]:
quotes = soup2.find_all('div', {'class' : 'quote'})

In [57]:
quotes[0]

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

In [58]:
quotes[0].find('span', {'class' : 'text'}).text

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [59]:
quotes[0].find('small', {'class' : 'author'}).text

'Albert Einstein'

In [75]:
quotes[0].find('meta', {'class' : 'keywords'}).get('content').split(',')

['change', 'deep-thoughts', 'thinking', 'world']

In [79]:
# для одной страницы
for quote in quotes:
    q = quote.find('span', {'class' : 'text'}).text
    author = quote.find('small', {'class' : 'author'}).text
    tags = quote.find('meta', {'class' : 'keywords'}).get('content')


In [86]:
# для 10 страниц
# Переменная для хранения цитат

qts = []

for i in tqdm(range(1,11)):
    page = requests.get(f'https://quotes.toscrape.com/page/{i}/')
    soup = BeautifulSoup(page.text, 'html')
    quotes = soup.find_all('div', {'class' : 'quote'})
    for quote in quotes:
        q = quote.find('span', {'class' : 'text'}).text
        author = quote.find('small', {'class' : 'author'}).text
        tags = quote.find('meta', {'class' : 'keywords'}).get('content')
        qts.append((q,author,tags))
#     time.sleep(3)
#print(qts)


    


100%|██████████| 10/10 [00:08<00:00,  1.15it/s]

[('“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'Albert Einstein', 'change,deep-thoughts,thinking,world'), ('“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'J.K. Rowling', 'abilities,choices'), ('“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'Albert Einstein', 'inspirational,life,live,miracle,miracles'), ('“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'Jane Austen', 'aliteracy,books,classic,humor'), ("“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”", 'Marilyn Monroe', 'be-yourself,inspirational'), ('“Try not to become a man of success. Rather become a man of value.”', 'Albert Einstein', 'adulthood,success,value'), ('“It is better to be hated for what you are than t

In [102]:
tag = input('Give me a tag: ')

Give me a tag: love


In [110]:
for q in qts:
    if tag in q[2]:
        print(q[0],',',q[1], end='\n----------------------------------------------------\n')

“It is better to be hated for what you are than to be loved for what you are not.” , André Gide
----------------------------------------------------
“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never find that half who makes you whole and that goes for everything. Just because you fail once, doesn't mean you're gonna fail at everything. Keep trying, hold on, and alway